# Lab - Making predictions with logistic regression
1. Create a query or queries to extract the information you think may be relevant for building the prediction model. It should include some film features and some rental features (X).
2. Create a query to get the list of all unique film titles and a boolean indicating if it was rented (rental_date) in May 2005. (Create new column called - 'rented_in_may'). This will be our TARGET (y) variable.
3. Read the data into a Pandas dataframe. At this point you should have 1000 rows. Number of columns depends on the number of features you chose.
4. Analyze extracted features (X) and transform them. You may need to encode some categorical variables, or scale numerical variables.
5. Create a logistic regression model to predict 'rented_in_may' from the cleaned data.

In [1]:
import pandas as pd
import numpy as np

In [2]:
import pymysql
from sqlalchemy import create_engine
import getpass  # To get the password without showing the input
password = getpass.getpass()

········


In [4]:
#1 
connection_string = 'mysql+pymysql://root:' + password + '@localhost/sakila'
engine = create_engine(connection_string)

In [5]:
#using read table
pd.read_sql_table('inventory',engine)

/Users/Anne-so1/anaconda3/lib/python3.11/site-packages/pandas/io/sql.py:1627: SAWarning: Did not recognize type 'geometry' of column 'location'
  self.meta.reflect(bind=self.con, only=[table_name])


,inventory_id,film_id,store_id,last_update
0,1,1,1,2006-02-15 05:09:17
1,2,1,1,2006-02-15 05:09:17
2,3,1,1,2006-02-15 05:09:17
3,4,1,1,2006-02-15 05:09:17
4,5,1,2,2006-02-15 05:09:17
...,...,...,...,...
4576,4577,1000,1,2006-02-15 05:09:17
4577,4578,1000,2,2006-02-15 05:09:17
4578,4579,1000,2,2006-02-15 05:09:17
4579,4580,1000,2,2006-02-15 05:09:17


In [24]:
#decided not to keep "original_language_id" because there were lots of nulls
#count distinct to avoid overcounting in the inventory
query = '''SELECT 
    f.film_id,
    f.title,
    f.release_year,
    f.language_id,
    f.rental_duration,
    f.length, 
    f.rental_rate,
    f.rating,
    f.replacement_cost,
    fc.category_id,
    COUNT(DISTINCT i.inventory_id) AS num_inventory_may,
    COUNT(r.rental_id) AS num_rentals_may
FROM 
    film f
LEFT JOIN 
    inventory i ON f.film_id = i.film_id
LEFT JOIN 
    rental r ON i.inventory_id = r.inventory_id
LEFT JOIN
    film_category fc ON f.film_id=fc.film_id
WHERE 
    (r.rental_date IS NULL) OR (r.rental_date >= '2005-05-01' AND r.rental_date < '2005-06-01')
GROUP BY 
    f.film_id, f.title, f.release_year, f.language_id, f.rental_duration, 
    f.length, f.rental_rate, f.rating, f.replacement_cost, fc.category_id;'''

data = pd.read_sql_query(query, engine)



In [25]:
data.head(10)
#data.describe().T

,film_id,title,release_year,language_id,rental_duration,length,rental_rate,rating,replacement_cost,category_id,num_inventory_may,num_rentals_may
0,1,ACADEMY DINOSAUR,2006,1,6,86,0.99,PG,20.99,6,3,2
1,3,ADAPTATION HOLES,2006,1,7,50,2.99,NC-17,18.99,6,1,1
2,4,AFFAIR PREJUDICE,2006,1,5,117,2.99,G,26.99,11,2,2
3,5,AFRICAN EGG,2006,1,6,130,2.99,G,22.99,8,1,1
4,6,AGENT TRUMAN,2006,1,3,169,2.99,PG,17.99,9,2,2
5,8,AIRPORT POLLOCK,2006,1,6,54,4.99,R,15.99,11,3,3
6,11,ALAMO VIDEOTAPE,2006,1,6,126,0.99,G,16.99,9,2,2
7,12,ALASKA PHANTOM,2006,1,6,136,0.99,PG,22.99,12,2,2
8,14,ALICE FANTASIA,2006,1,6,94,0.99,NC-17,23.99,4,0,0
9,15,ALIEN CENTER,2006,1,5,46,2.99,NC-17,10.99,9,2,2


In [27]:
data.isna().sum()

film_id              0
title                0
release_year         0
language_id          0
rental_duration      0
length               0
rental_rate          0
rating               0
replacement_cost     0
category_id          0
num_inventory_may    0
num_rentals_may      0
dtype: int64

In [48]:
#Create new column called 'rented_in_may'
query = '''SELECT 
    f.film_id,
    f.title,
    f.release_year,
    f.language_id,
    f.rental_duration,
    f.length, 
    f.rental_rate,
    f.rating,
    f.replacement_cost,
    fc.category_id,
    COUNT(DISTINCT i.inventory_id) AS num_inventory_may,
    COUNT(r.rental_id) AS num_rentals_may,
    max(CASE WHEN r.rental_date BETWEEN '2005-05-01' AND '2005-05-31' THEN 1 ELSE 0 END) AS rented_in_may
FROM 
    film f
LEFT JOIN 
    inventory i ON f.film_id = i.film_id
LEFT JOIN 
    rental r ON i.inventory_id = r.inventory_id
LEFT JOIN
    film_category fc ON f.film_id=fc.film_id   
GROUP BY 
    f.film_id, f.title, f.release_year, f.language_id, f.rental_duration, 
    f.length, f.rental_rate, f.rating, f.replacement_cost, fc.category_id;'''

data = pd.read_sql_query(query, engine)
data.head(3)
#data.shape

,film_id,title,release_year,language_id,rental_duration,length,rental_rate,rating,replacement_cost,category_id,num_inventory_may,num_rentals_may,rented_in_may
0,1,ACADEMY DINOSAUR,2006,1,6,86,0.99,PG,20.99,6,8,23,1
1,2,ACE GOLDFINGER,2006,1,3,48,4.99,G,12.99,11,3,7,0
2,3,ADAPTATION HOLES,2006,1,7,50,2.99,NC-17,18.99,6,4,12,0


In [49]:
data.dtypes

film_id                int64
title                 object
release_year           int64
language_id            int64
rental_duration        int64
length                 int64
rental_rate          float64
rating                object
replacement_cost     float64
category_id            int64
num_inventory_may      int64
num_rentals_may        int64
rented_in_may          int64
dtype: object

In [37]:
data['language_id'].value_counts(dropna=False)

data['rating'].value_counts(dropna=False)

rating
PG-13    165
NC-17    150
PG       140
R        140
G        133
Name: count, dtype: int64

# Transform variables
You may need to encode some categorical variables, or scale numerical variables.

In [55]:
##encoding FILM CAT##
from sklearn.preprocessing import OneHotEncoder
#rating=data[['rating']]
encoder=OneHotEncoder().fit(data[['category_id']])
encoded=encoder.transform(data[['category_id']]).toarray()

rating_encoded=pd.DataFrame(encoded,
                            columns=encoder.get_feature_names_out(['category_id']))
rating_encoded

,category_id_1,category_id_2,category_id_3,category_id_4,category_id_5,category_id_6,category_id_7,category_id_8,category_id_9,category_id_10,category_id_11,category_id_12,category_id_13,category_id_14,category_id_15,category_id_16
0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
996,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
997,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
998,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [56]:
##encoding RATING##
from sklearn.preprocessing import OneHotEncoder
#rating=data[['rating']]
encoder=OneHotEncoder().fit(data[['rating']])
encoded=encoder.transform(data[['rating']]).toarray()

rating_encoded=pd.DataFrame(encoded,
                            columns=encoder.get_feature_names_out(['rating']))
rating_encoded

,rating_G,rating_NC-17,rating_PG,rating_PG-13,rating_R
0,0.0,0.0,1.0,0.0,0.0
1,1.0,0.0,0.0,0.0,0.0
2,0.0,1.0,0.0,0.0,0.0
3,1.0,0.0,0.0,0.0,0.0
4,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...
995,1.0,0.0,0.0,0.0,0.0
996,0.0,1.0,0.0,0.0,0.0
997,0.0,1.0,0.0,0.0,0.0
998,0.0,0.0,0.0,0.0,1.0


# Train test

In [54]:
##x and y##
y = data['rented_in_may'] #'rented_in_may' is our (y) target
X = data.drop(['rented_in_may','title'], axis=1) #X is all the other columns
##train test split##
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=100)

#features = pd.DataFrame(xx['data'], columns = xx['feature_names'])
#labels = pd.Series(xx['target'], name = 'labels')

# Scaling min max

In [ ]:
#With train data
from sklearn.preprocessing import MinMaxScaler

X_train_num = X_train.select_dtypes(include = np.number)

# Scaling 
transformer = MinMaxScaler().fit(X_train_num) # need to keep transformer
X_train_normalized = transformer.transform(X_train_num)
X_train_norm = pd.DataFrame(X_train_normalized, columns=X_num.columns)
X_train_norm

In [ ]:
#With dta 
from sklearn.preprocessing import MinMaxScaler

X_num = data.select_dtypes(include = np.number)
X_cat = data.select_dtypes(include = object)

# Scaling data
transformer = MinMaxScaler().fit(X_num)
x_normalized = transformer.transform(X_num)
x_norm = pd.DataFrame(x_normalized, columns=X_num.columns)
x_norm.head()

# Plotting

In [ ]:
import matplotlib.pyplot as plt
corr_matrix=data.select_dtypes('number').corr()  
fig, ax = plt.subplots(figsize=(10, 8))
ax = sns.heatmap(corr_matrix, annot=True)
plt.show()

In [ ]:
sns.displot(X_train_norm, kde=True)
plt.show()

In [5]:
from sklearn.linear_model import LogisticRegression #log reg will guess already 1 and 0 
LRmodel = LinearRegression()
LRmodel = LRmodel.fit(X_train, y_train)
LRmodel.score(X_test,y_test)

0.6844267283527068

In [8]:
X_train_categorical = X_train.select_dtypes(include = object)
X_train_cat = pd.get_dummies(X_train_categorical, 
                             columns=['type', 'operation', 'k_symbol', 'duration'],
                             drop_first=True)
#get_dummies Convert categorical variable into dummy/indicator variables.
#Each variable is converted in as many 0/1 variables as there are different values. Columns in the output are each named after a value; if the input is a DataFrame
X_train_cat.head()